In [ ]:
from retry import retry
import requests
from bs4 import BeautifulSoup
import pandas as pd

# !pip install retry

In [ ]:
# 東京23区
base_url = "https://suumo.jp/jj/chintai/ichiran/FR301FC001/?fw2=&mt=9999999&cn=9999999&ta=13&et=9999999&sc=13105&shkr1=03&ar=030&bs=040&ct=9999999&shkr3=03&shkr2=03&mb=0&shkr4=03&cb=0.0&page={}"
# https://suumo.jp/jj/chintai/ichiran/FR301FC005/?fw2=&mt=9999999&cn=9999999&ta=13&et=9999999&sc=13105&shkr1=03&ar=030&bs=040&ct=9999999&shkr3=03&shkr2=03&mb=0&shkr4=03&cb=0.0&&page={}
@retry(tries=3, delay=10, backoff=2)
def get_html(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "html.parser")
    return soup

In [ ]:
all_data = []
max_page = 1

for page in range(1, max_page+1):
    # define url
    url = base_url.format(page)

    # get html
    soup = get_html(url)

    # extract all items
    items = soup.findAll("div", {"class": "cassetteitem"})
    print("page", page, "items", len(items))

    # process each item
    for item in items:
        stations = item.findAll("div", {"class": "cassetteitem_detail-text"})

        # process each station
        for station in stations:
            # define variable
            base_data = {}

            # collect base information
            base_data["名称"] = item.find("div", {"class": "cassetteitem_content-title"}).getText().strip()
            base_data["カテゴリー"] = item.find("div", {"class": "cassetteitem_content-label"}).getText().strip()
            base_data["アドレス"] = item.find("li", {"class": "cassetteitem_detail-col1"}).getText().strip()
            base_data["アクセス"] = station.getText().strip()
            base_data["築年数"] = item.find("li", {"class": "cassetteitem_detail-col3"}).findAll("div")[0].getText().strip()
            base_data["構造"] = item.find("li", {"class": "cassetteitem_detail-col3"}).findAll("div")[1].getText().strip()

            # process for each room
            tbodys = item.find("table", {"class": "cassetteitem_other"}).findAll("tbody")

            for tbody in tbodys:
                data = base_data.copy()

                data["階数"] = tbody.findAll("td")[2].getText().strip()

                data["家賃"] = tbody.findAll("td")[3].findAll("li")[0].getText().strip()
                data["管理費"] = tbody.findAll("td")[3].findAll("li")[1].getText().strip()

                data["敷金"] = tbody.findAll("td")[4].findAll("li")[0].getText().strip()
                data["礼金"] = tbody.findAll("td")[4].findAll("li")[1].getText().strip()

                data["間取り"] = tbody.findAll("td")[5].findAll("li")[0].getText().strip()
                data["面積"] = tbody.findAll("td")[5].findAll("li")[1].getText().strip()

                data["URL"] = "https://suumo.jp" + tbody.findAll("td")[8].find("a").get("href")

                all_data.append(data)

# convert to dataframe
df = pd.DataFrame(all_data)

page 1 items 30


In [ ]:
df.to_csv("./output/rooms/test.csv", encoding = 'utf-8-sig', index = 0)

In [ ]:
#urls = ['https://suumo.jp/chintai/jnc_000088712304/?bc=100367779141', ]
urls = df["URL"].tolist()[:10]
room_details = []

for url in urls:
    data = {"URL": url}

    # get html
    soup = get_html(url)

    # 部屋情報
    room_info = soup.findAll("table", {"class": "property_view_table"})[0]
    titles = [x.getText().strip() for x in room_info.findAll("th", {"class": "property_view_table-title"})]
    values = [x.getText().strip() for x in room_info.findAll("td", {"class": "property_view_table-body"})]
    for i in range(len(titles)):
        data[titles[i]] = values[i]

    # 部屋の特徴・設備
    data["部屋の特徴・設備"] = soup.find("div", {"class": "bgc-wht ol-g"}).find("li").getText().strip()

    # 物件概要
    room_info = soup.findAll("table", {"class": "data_table table_gaiyou"})[0]
    titles = [x.getText().strip() for x in room_info.findAll("th")]
    values = [x.getText().strip() for x in room_info.findAll("td")]
    for i in range(len(titles)):
        data[titles[i]] = values[i]

    room_details.append(data)
    #print(len(titles))
    #print(len(values))
    #print(x)
    #print(titles, values)

df_details = pd.DataFrame(room_details)

In [ ]:
df_details.to_csv("./output/details/test.csv", encoding = 'utf-8-sig', index = 0)

In [ ]:
df.head(1)

,名称,カテゴリー,アドレス,アクセス,築年数,構造,階数,家賃,管理費,敷金,礼金,間取り,面積,URL
0,東京メトロ丸ノ内線 茗荷谷駅 3階建 築7年,賃貸マンション,東京都文京区小日向３,東京メトロ丸ノ内線/茗荷谷駅 歩6分,築7年,3階建,2階,15.2万円,7000円,15.2万円,30.4万円,1LDK,38.76m2,https://suumo.jp/chintai/jnc_000088712304/?bc=...
